In [1]:
import numpy as np
import pandas as pd
import annoy
import pickle
import re
import lightfm
import scipy
import tqdm
from tqdm import tqdm_notebook

import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import FastText
from gensim.models import Word2Vec

from sklearn.model_selection import train_test_split

C:\Users\Lyo\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [2]:
SIZE = length_vector = 20

In [3]:
DATADIR = 'data'

Загружаем датасеты

In [4]:
# Чеки
checks = pd.read_csv('/'.join([DATADIR, 'чековые данные.csv']))

# Информация о товарах
with open('/'.join([DATADIR, 'Product_dict.pkl']), 'rb') as file:
    products = pickle.load(file)

C:\Users\Lyo\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3186: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Посмотрим на данные

In [5]:
checks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000003 entries, 0 to 20000002
Data columns (total 9 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   sale_date_date           object 
 1   contact_id               object 
 2   shop_id                  float64
 3   product_id               float64
 4   name                     object 
 5   product_sub_category_id  float64
 6   product_category_id      float64
 7   brand_id                 float64
 8   quantity                 object 
dtypes: float64(5), object(4)
memory usage: 1.3+ GB


In [6]:
checks.head()

,sale_date_date,contact_id,shop_id,product_id,name,product_sub_category_id,product_category_id,brand_id,quantity
0,2018-12-07,1260627,1455.0,168308.0,(197312) Пакет-майка 25см х 45см,906.0,205.0,-1.0,"1,00"
1,2018-12-07,198287,279.0,134832.0,(62448) Перекись водорода р-р наружн. 3% фл.по...,404.0,93.0,-1.0,"1,00"
2,2018-12-07,2418385,848.0,101384.0,(72183) Салициловая кислота р-р спирт 2% фл 40...,404.0,93.0,-1.0,"1,00"
3,2018-12-07,1285774,1511.0,168570.0,(197309) Пакет 28см х 50см,906.0,205.0,-1.0,"1,00"
4,2018-12-07,1810323,1501.0,168319.0,(197310) Пакет 30см х 60см,906.0,205.0,-1.0,"1,00"


In [7]:
pd.DataFrame.from_dict(products, orient='index').head()

,0
168308,(197312) Пакет-майка 25см х 45см 906
134832,(62448) Перекись водорода р-р наружн. 3% фл.по...
101384,(72183) Салициловая кислота р-р спирт 2% фл 40...
168570,(197309) Пакет 28см х 50см 906
146960,"(111023) Пакет ""Аптека Озерки"" 28 х 35см 906"


In [8]:
# Удалим NA
checks.dropna(inplace=True)
# В конце файла левые записи, отфильтруем
checks = checks[checks.sale_date_date.str.startswith('20')]

In [9]:
# Преобразуем sale_date_date
checks.sale_date_date=pd.to_datetime(checks.sale_date_date)

In [10]:
# Наименование содержится в обоих датасетах.
checks.drop(['name'], axis=1, inplace=True)

In [11]:
# Конвертируем quantity в float
checks['quantity'] = checks['quantity'].apply(lambda x: float(x.replace(',', '.')))
# Остальные в int
checks['shop_id'] = checks['shop_id'].apply(int)
checks['product_id'] = checks['product_id'].apply(int)
checks['product_category_id'] = checks['product_category_id'].apply(int)

In [12]:
# Сформируем новый признак на основе первых трёх колонок (с учётом даты продажи)
checks['group_column'] = checks.sale_date_date.apply(str) + "_" + checks.contact_id.apply(str) + "_" + checks.shop_id.apply(str)
checks

,sale_date_date,contact_id,shop_id,product_id,product_sub_category_id,product_category_id,brand_id,quantity,group_column
0,2018-12-07,1260627,1455,168308,906.0,205,-1.0,1.0,2018-12-07 00:00:00_1260627_1455
1,2018-12-07,198287,279,134832,404.0,93,-1.0,1.0,2018-12-07 00:00:00_198287_279
2,2018-12-07,2418385,848,101384,404.0,93,-1.0,1.0,2018-12-07 00:00:00_2418385_848
3,2018-12-07,1285774,1511,168570,906.0,205,-1.0,1.0,2018-12-07 00:00:00_1285774_1511
4,2018-12-07,1810323,1501,168319,906.0,205,-1.0,1.0,2018-12-07 00:00:00_1810323_1501
...,...,...,...,...,...,...,...,...,...
19999995,2018-06-13,1601618,1499,66842,615.0,140,1838.0,1.0,2018-06-13 00:00:00_1601618_1499
19999996,2018-06-13,1394104,1495,136795,738.0,170,-1.0,1.0,2018-06-13 00:00:00_1394104_1495
19999997,2018-06-13,1570654,1516,119513,738.0,170,-1.0,1.0,2018-06-13 00:00:00_1570654_1516
19999998,2018-06-13,1924036,1485,71723,637.0,146,-1.0,1.0,2018-06-13 00:00:00_1924036_1485


In [13]:
checks.sort_values('sale_date_date', inplace=True)

In [14]:
# Валидационная и тестовая выборка
train, test = train_test_split(checks, test_size=0.2, shuffle=False)

In [15]:
train.head()

,sale_date_date,contact_id,shop_id,product_id,product_sub_category_id,product_category_id,brand_id,quantity,group_column
4744563,2018-01-01,127175,603,153822,439.0,101,-1.0,1.0,2018-01-01 00:00:00_127175_603
4773074,2018-01-01,1786329,1482,168570,906.0,205,-1.0,1.0,2018-01-01 00:00:00_1786329_1482
6689191,2018-01-01,1326823,1522,112760,577.0,130,-1.0,10.0,2018-01-01 00:00:00_1326823_1522
6689190,2018-01-01,1556808,1530,164413,762.0,176,2738.0,4.0,2018-01-01 00:00:00_1556808_1530
6689189,2018-01-01,1383494,1516,123127,404.0,93,-1.0,4.0,2018-01-01 00:00:00_1383494_1516


#### Контентные рекомендации

Препроцессинг текста

In [16]:
exclude = set(string.punctuation)
stop_w = set(get_stop_words(language='ru'))
morpher = MorphAnalyzer()

In [17]:
# Компилируем шаблоны для быстрой обработки
symbols_pattern = re.compile(pattern = "["
                            "@_!#№$%^&*()<>?/\|}{~:√•"
                            "]+", flags = re.UNICODE)
single_pattern = re.compile(r'\s+[a-zA-Zа-яА-Я]\s+')
# Двойные пробелы
space_pattern = re.compile(r'\s+')

In [18]:
def clear_text(text):
    """ Функция удаления спецсимволов"""
    # Удаление спецсимволов
    text = re.sub(r'\W', ' ', str(text))
    text = symbols_pattern.sub(r' ', str(text))
    
    # Удалим все цифры
    text = re.sub(r'\d', ' ', str(text))

    # Одиночные буквы
    text = single_pattern.sub(r' ', text)

    # Двойные пробелы
    text = space_pattern.sub(' ', text)
    
    return text


In [19]:
def preprocess_text(text):
    """ Обработка текста """
    # srip + lower + punctuation
    sentence = ''.join([x for x in str(text).strip().lower() if x not in exclude])
    
    # Очистка
    sentence = clear_text(sentence)
    
    # Лемматизация и стопслова
    sentence = ' '.join([morpher.parse(word)[0].normal_form for word in sentence.split() if word not in stop_w])
    return [x for x in sentence.split() if len(x)>2]

In [20]:
# Обрабатываем наименования товаров
sentences = [preprocess_text(v) for k,v in products.items()]

In [21]:
sentences[:5]

[['пакетмайк', 'смотреть', 'смотреть'],
 ['перекись', 'водород', 'наружн', 'флполимерна'],
 ['салициловый', 'кислота', 'спирт'],
 ['пакет', 'смотреть', 'смотреть'],
 ['пакет', 'аптека', 'озерко', 'смотреть']]

In [22]:
# Обучим Fasttext
modelFT = FastText(sentences=sentences, size=SIZE, min_count=1, window=5)

In [23]:
with open('/'.join([DATADIR, 'modelFT.dump']), 'wb') as file:
    pickle.dump(modelFT, file)

In [24]:
def get_vector(sent, model):
    """ Функция возвращает вектор предложения """
    n_w2v = 0
    vector = np.zeros(length_vector)
    for word in sent:
        if word in model.wv:
            vector += model.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    return vector

In [25]:
# Строим индексы
index = annoy.AnnoyIndex(SIZE, 'angular')

index_map = {}
counter = 0

for k, v in products.items():
    index_map[counter] = (k, v)
    
    index.add_item(counter, get_vector(clear_text(v), modelFT)) 
    counter += 1

index.build(10)

True

In [26]:
def get_recommendation(items:list):
    """ Функция возвращает рекомендованный список товаров """
    # Количество товаров
    cnt = len(items)
    
    vector = np.zeros(length_vector)
    # Усредним все векторы
    for id in items:
        vector += get_vector(clear_text(products[f'{id}']), modelFT)
    # Взвращаем в виде (id, name), исключая оригинальное совпадение
    return [(index_map[x][0], products[index_map[x][0]]) for x in index.get_nns_by_vector(vector/cnt, 10) if int(index_map[x][0]) not in items]

In [27]:
# Если интересуется одним товаром
prod_id = 110629
print('Интересует:', products[f'{prod_id}'])
print('Рекомендации на основе content-based (варианты):') 
get_recommendation([prod_id])

Интересует: (57733) Корвалол капли д/приема внутрь 25мл 738
Рекомендации на основе content-based (варианты):


[('165144', '(195223) Корвалол капли д/приема внутрь 25мл 738'),
 ('54295', '(40304) Корвалол капли д/приема внутрь 25мл 738'),
 ('111555', '(44903) Корвалол капли д/приема внутрь 15мл -1'),
 ('106107', '(116503) Корвалол капли д/приема внутрь 50мл -1'),
 ('159605', '(182746) Корвалол капли д/приема внутрь 50мл 631'),
 ('24714', '(97167) Корвалол капли д/приема внутрь 50мл -1'),
 ('54621', '(65447) Корвалол капли д/приема внутрь 50мл 738'),
 ('67170', '(61374) Корвалол капли д/приема внутрь 25мл 738'),
 ('124458', '(42890) Корвалол капли д/приема внутрь 25мл 738')]

Модель предложила близкие по значению варианты, дозировки. 

In [28]:
# Серия товаров
# В функцию get_recommendation мы заложили возможность анализа списка товаров
get_recommendation([158600, 143709])

[('192620',
  '(194901) Медик-о-Планет Шприц одноразовый 3-комп.с иглой 21G 0,8х40мм 20мл №1 762'),
 ('192505',
  '(194900) Медик-о-Планет Шприц одноразовый 3-комп.с иглой 23G (0,6х30мм) 3мл №1 762'),
 ('101037', '(33021) Пирацетам р-р д/и 20% амп/поддон 5мл N10 -1'),
 ('153870',
  '(122362) Виши ЛифтАктив Супрем крем д/нормальной кожи 50мл + термальная вода 50мл (подарок) 679'),
 ('51207',
  '(58653) Раптор Пластины Новая Формула от комаров б/запаха в мини-прилавке арт.D1823M 561'),
 ('82297',
  '(102920) ГринФарма ФармаВолюм шампунь для объема для тонких и хрупких волос 500мл -1'),
 ('163411', '(186147) Ляпко Аппликатор одинарный (6,2мм иглы) 407'),
 ('150352', '(15583) Папаверина гидрохлорид р-р 2% амп 2мл N10 397'),
 ('130332', '(113736) Папаверина гидрохлорид р-р 2% амп 2мл N10 397'),
 ('79736', '(61049) Папаверина гидрохлорид р-р 2% амп 2мл N10 397')]

In [29]:
# Обучение эмбеддингов по последовательности
train['product_id'] = train['product_id'].apply(str)
grouped = train.groupby('group_column')
sentences = []

In [30]:
# Сформируем последовательности
sentences = []
for group in tqdm_notebook(grouped.groups):
    items = grouped.get_group(group)['product_id'].values
    if len(items) < 3:
        continue
    sentences.append(list(items))

C:\Users\Lyo\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/4786198 [00:00<?, ?it/s]

In [31]:
sentences[0]

['62087', '120171', '69028']

In [32]:
# Обучим модель Word2Vec
modelW2V = Word2Vec(sentences, size=SIZE)

In [33]:
with open('/'.join([DATADIR, 'modelW2V.dump']), 'wb') as file:
    pickle.dump(modelW2V, file)

In [34]:
def recommend_w2v(items_list):
    """ Функция возвращает рекомендации анализируя последовательность"""
    current_vector = get_vector(items_list, modelW2V)
    return [products[i[0]] for i in modelW2V.similar_by_vector(current_vector, 10)]

In [35]:
# В корзине
[products[x] for x in ['99821', '138583', '45321', '134475']]

['(103007) Билобил интенс 120 капс.№60 639',
 '(112042) Дузофарм таб. п.п.о.50мг №90 736',
 '(25405) Эриус сироп фл.60мл №1 395',
 '(41834) Аскорил Экспекторант сироп 100мл 550']

In [36]:
# Рекомендации
recommend_w2v(['99821', '138583', '45321', '134475'])

C:\Users\Lyo\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).
  after removing the cwd from sys.path.


['(121015) Момат Рино спрей наз.доз.50мкг/доза 120доз фл.с доз. 459',
 '(110018) Дезринит спрей наз.доз.50мкг/доза фл.18г 395',
 '(43714) Флуифорт 9гр/120мл сироп фл N1 700',
 '(105792) Назонекс наз спрей 50мкг/доза 60доз N1 701',
 '(2710) Тафен назаль спрей наз. доз. 50 мкг/доза 200 доз. 395',
 '(65551) Авамис спрей назальный 27,5 мкг/доза 120 доз 395',
 '(119001) Аква Марис Эктоин спрей назальный фл.20мл №1 701',
 '(47968) Ремо-вакс ушные капли 10 мл. 697',
 '(112733) Момат Рино Адванс спрей наз.доз.140мкг+50мкг/доза 150доз фл.с доз. 550',
 '(103962) Назонекс наз спрей 50мкг/доза 120доз N1 701']

#### Двухстадийная рекомендательная система

Кандидатогенераторы
1. Похожие товары по наименованию
2. Товары по группам
3. Рандомные товары

In [37]:
# Подготовим dict для быстрого поиска групп и товаров одной группы
products_category = {}
category_dict = {}

for product in [int(x) for x in products]:
    groups = list(checks[checks.product_id == product].product_category_id.unique())
    products_category[product] = groups
    for group in groups:
        if group not in category_dict:
            category_dict[group] = []
        category_dict[group].append(product)

In [38]:
with open('products_category.dump', 'wb') as file:
    pickle.dump(products_category, file)

with open('category_dict.dump', 'wb') as file:
    pickle.dump(category_dict, file)


Создадим метод, который возвращает случайные варианты из предложенного списка. 5 шт достаточно.

In [39]:
def rand_recommendation(samples, num_res=5)->list:
    """ Возвращает случайный список """
    assert len(samples) >= num_res
    return list(np.random.choice(samples, num_res, replace=False))

In [40]:
def candidates_generator(item_id) -> list:
    """ Функция генерирует кандидаты на основе id товара """
    candidates = []
    if item_id is not None:
        # Кандидаты по наименованию
        candidates = [x[0] for x in get_recommendation([item_id])]
        
        # Кандидаты из той же группы товаров
        # Выясняем группу
        group_id = products_category[item_id]
        # Получаем список товаров из этой же группы
        candidates_group = rand_recommendation(category_dict[group_id[0]])
        
        # Случайный товар из доступного (как вариант, фильтровать ешё по shop_id)
        candidates_random = rand_recommendation([x for x in products])
        
        candidates.extend(candidates_group)
        candidates.extend(candidates_random)
        return list(set([int(x) for x in candidates if x != item_id]))


In [41]:
# Сгенерируем кандидаты для product_id=72436
candidates_generator(72436)

[24705,
 94465,
 98948,
 136196,
 142097,
 134848,
 157122,
 350658,
 58451,
 61026,
 217957,
 101993,
 121833,
 38251,
 149228,
 110956,
 86771,
 61812,
 62328]

Ранжирование будем производить на основе товара и магазина, где находится покупатель

Выделим популярность товара

In [42]:
df = checks[['product_id', 'shop_id', 'quantity']]
df[['product_id', 'quantity']].groupby(['product_id']).sum().sort_values(by='quantity', ascending=False).describe()

,quantity
count,36112.000000
mean,773.323190
std,6463.368434
min,0.015000
25%,2.000000
50%,16.000000
75%,199.000000
max,565983.450000


Условно, будем считать, что значение quantity > 75% - очень популярно, выставим флаг score=1

In [43]:
ratings = df.groupby(['product_id', 'shop_id']).sum()
ratings.reset_index(inplace=True)
ratings

,product_id,shop_id,quantity
0,8086,43,13.0
1,8086,51,11.0
2,8086,52,1.0
3,8086,53,1.0
4,8086,57,16.0
...,...,...,...
2373060,362230,363,1.0
2373061,362418,1899,1.0
2373062,362464,313,3.0
2373063,363111,454,1.0


In [44]:
# "0 < 75% < 1"
perc_75 = np.percentile(ratings.quantity, 75)
ratings['score'] = (ratings['quantity'] > perc_75).apply(int)

In [45]:
# Подготовим ранжирующие факторизационные машины
train_pivot = (1 + ratings[['shop_id', 'product_id', 'score']].pivot(index='shop_id', columns='product_id', values='score')).fillna(0)
train_pivot.reset_index(inplace=True, drop=False)

In [46]:
# Сформируем индекс магазинов в матрице
shops_index = {train_pivot.iloc[x]['shop_id']: x for x in range(train_pivot.shape[0])}
# Сформируем индекс товаров в матрице
products_index = {train_pivot.columns[x]: x for x in range(train_pivot.shape[1])}

In [47]:
# Модель работает с разреженными матрицами
train_pivot = scipy.sparse.csr_matrix(train_pivot)

In [48]:
# Обучаем модель
model = lightfm.LightFM(loss='warp')
model.fit(train_pivot, epochs=30)

In [49]:
def get_shop_index(item_id: int):
    """ Возвращает индекс для модели по id магазина """
    return shops_index[item_id]

In [50]:
def get_product_index(item_id: int):
    """ Возвращает индекс для модели по id товара """
    return products_index[item_id]

In [51]:
def get_recom_from_shop(shop_id: int, product_id: int) -> list:
    """ Функция генерирует предложения на основе выбранного товара и магазина, где находится покупатель """
    candidates_of_products = candidates_generator(product_id)
    candidates_of_products_by_index = [*map(get_product_index, candidates_of_products)]
    recommendations = model.predict(get_shop_index(shop_id), candidates_of_products_by_index)
    # Сортируем
    recommendations = sorted(zip(candidates_of_products, recommendations), key=lambda x: x[1], reverse=True)
    return [products[str(x[0])] for x in recommendations]

In [52]:
# Допустим, что человек в магазине с shop_id=1899, покупает товар с product_id=72436
get_recom_from_shop(1899, 72436)

['(65902) Калия хлорид р-р д/и 4% амп 10мл N10 437',
 '(1676) Крем-сыр "Рама" бонжур с марин огурч. 200г -1',
 '(118240) Арнебия Магний+ Витамин С таб.шип.№20 427',
 '(70656) Солгар Кардио Саппорт плюс таб. 120мг №60 428',
 '(109499) Калия хлорид р-р д/и 4% амп 10мл N10 437',
 '(25325) Калия хлорид р-р д/и 4% амп 10мл N10 437',
 '(64358) Солгар Коэнзим Q-10 капс. 30мг №30 428',
 '(179909) Калия хлорид р-р д/и 4% амп 10мл №20 738',
 '(120238) Массажер для тела игольчатый (М-403) 935',
 '(111901) Калия хлорид р-р д/и 4% амп 10мл N10 437',
 '(112756) Листерин ополаскиватель д/полости рта Зеленый Чай 250мл 537',
 '(102459) КомплигамВ Комплекс таб. №30 430',
 '(115635) Калия хлорид р-р д/и 4% амп 10мл N10 437',
 '(71304) Кальция хлорид р-р д/и 10% амп 5мл N10 437',
 '(106666) Калия хлорид р-р д/и 4% амп 10мл N10 437',
 '(88447) Арпефлю таб.п.п.о.50мг №10 -1',
 '(44815) Калия хлорид р-р д/и 4% амп 10мл N10 437',
 '(54255) Левзея П № 100 табл. 441',
 '(59512) Ингалятор компрессорный CN-231 58